# Assignment 2 (100 points)

You are expected to complete this notebook with lines of code, plots and texts. You might need to create new cells with original code or text for your analyses. This assignment has a total of 100 points.

For assignment submission, you will submit this notebook file (.ipynb) on Canvas with cells executed and outputs visible. Your submitted notebook **must** follow these guidelines:
- No other dataset than the provided datasets should be used.
- Training, validation and testing splits should be the same as the ones provided.
- The cell outputs in your delivered notebook should be reproducible.
- Printing out the evaluation metric evidence that your model achieves the evaluation requirement. Optionally, you can also add plot of the evaluation metric changing over the course of training process.
- Providing code associated with the conclusions you make in your analysis as well as code that is used to generated plot, images, etc. for your analysis.
- All code must be your own work. Code cannot be copied from external sources or another students. You may copy code from cells that are pre-defined in this notebook if you think it is useful to reuse in another question.
- All images must be generated from data generated in your code. Do **NOT** import/display images that are generated outside your code.
- Your analysis must be your own, but if you quote text or equations from another source please make sure to cite the appropriate references.
- Your input with code will be marked with comments ``###your code starts here###`` and ``###your code ends here###`` to specify where you need to write your code. You can also create a new code cell in between those marked comments.


**NOTES:**
- PyTorch needs to be downloaded and installed properly.
- You should use PyTorch 1.7 or later.
- If you need to import a different package than the ones already imported, please check with the TA if you can do so.
- Cells should be run in order, using Shift+Enter.
- Read all the provided code cells and comments as they contain variables and information that you may need to use to complete the notebook.
- To create a new text cell, select "+" button on the menu bar and change its type from "Code" to "Markdown".
- To modify a text cell, double click on it.
- More details on how to format markdown text can be found here: https://medium.com/ibm-data-science-experience/markdown-for-jupyter-notebooks-cheatsheet-386c05aeebed
- Your home directory on CADE machines has a small disk quota. It might be necessary, depending on how much your home directory is already occupied, to store the virtual environment inside a folder in ```/scratch/tmp/```. 
- **The accuracy requirement for each question is there to make sure you have performed sufficient amount of experiments to achieve a good result. Part of the grade is based on this.**

**Tips for training deep learning models:**
- Since the datasets being used here are small, you are probably going to have to use early stopping to prevent overfitting. This means that you will have to save your models in the middle of training. One of the ways to do so is to make a deep copy of it using ```copy.deepcopy``` function. 
- It is recommended to frequently monitor the behavior of the model at least once every epoch. You can either print out the training loss or evaluation metric of the training set to verify that the model is being optimized correctly. In this assignment, it should take somewhere between 1 and 20 epochs for a desired model to achieve the required accuracy.
- To search for the best hyperparameters for your model, it is usually better to start searching in a logarithmic scale. Usually power of 2 or 10 is used. 
- In https://medium.com/octavian-ai/which-optimizer-and-learning-rate-should-i-use-for-deep-learning-5acb418f9b2, an ablation study in finding optimal learning rates for different optimizers are listed. This could help in your search of optimal learning rate. For this assignment, you will probably get the best results with Adam optimizer and searching for the best learning rate in the range of learning rates provided in that article, which is from 0.00005 to 0.01 unless a question is asked to use a specific optimizer.
- Batch size seems to have a smaller impact than learning rate in the results. It should be enough if you test batch sizes between 8 and 32.
- We assume a GPU of at least 4GB of memory is available. If you want to try running the assignment with a GPU that has less than that, you can try changing the argument passed when calling the ```define_gpu_to_use``` function.  If you are getting out-of-memory errors for the GPU, you may want to check what is occupying the GPU memory by using the command ```!nvidia-smi```, which gives a usage report of the GPU. However, if you are using your own Windows machine, the nvidia-smi command used in the define_gpu_to_use function will not work. You can skip running this function but please check to make sure your GPU has a sufficient amount of free memory.
- For some of the questions, it might be useful for you to understand what the ResNet-18 PyTorch model is doing. You can have access to its source code here (https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py). The most important part you should know is the ```forward``` function from the ```ResNet``` class. 
- It might also be useful to print PyTorch models using ```print("model_name")```. This should give you a list of all the layers in the model.
- Here are a few PyTorch details not to forget:
    - Toggle train/eval mode for your model
    - Reset the gradients with ```zero_grad()``` before each call to ```backward()```
    - Check if the loss you are using receives logits or probabilities, and adapt your model output accordingly.
    - Reinstantiate your model every time you are starting a new training so that the weights are reset, if you plan to reuse the variable name.
    - Pass the model's parameters to the optimizer.

# Exercise 0 - Set-up Infrastructure (Total of 0 points)
## + Install Libraries:

In [1]:
!pip3 install -q kaggle
!pip3 install pydicom
!pip3 install scikit-learn

## + Import Libraries:

In [2]:
import getpass
import os
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import torchvision
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
import torch.nn.functional as F
import sys
from sklearn.metrics import roc_curve, auc
import copy
import torchvision.models as models
import tarfile
import time
from packaging import version
%matplotlib inline

##### Check Torch library requirement #####
my_torch_version = torch.__version__
minimum_torch_version = '1.7'
if version.parse(my_torch_version) < version.parse(minimum_torch_version):
    print('Warning!!! Your Torch version %s does NOT meet the minimum requirement!\
            Please update your Torch library\n' %my_torch_version)

## + Create Data Folder:

In [3]:
##### Check what kind of system you are using #####
try:
    hostname = !hostname
    if 'lab' in hostname[0] and '.eng.utah.edu' in hostname[0]:
        IN_CADE = True
        print("Cade running successfully")
    else:
        IN_CADE = False
except:
    IN_CADE = False

## Define the folders where datasets will be
machine_being_used = 'cade' if IN_CADE else ('other')
pre_folder = '/scratch/tmp/' if machine_being_used == 'cade' else './'
mnist_dataset_folder = pre_folder + 'deep_learning_datasets_ECE_6960_013/mnist'
xray14_dataset_folder = pre_folder + 'deep_learning_datasets_ECE_6960_013/chestxray14'

## Create directory if they haven't existed yet 
if not os.path.exists(mnist_dataset_folder):
    os.makedirs(mnist_dataset_folder)    
if machine_being_used != 'cade' and not os.path.exists(mnist_dataset_folder+'/MNIST'):        
    os.makedirs(mnist_dataset_folder+'/MNIST')
if machine_being_used != 'cade' and not os.path.exists(mnist_dataset_folder+'/MNIST/raw'):
    os.makedirs(mnist_dataset_folder+'/MNIST/raw')
if not os.path.exists(xray14_dataset_folder):
    os.makedirs(xray14_dataset_folder)

## + Request GPU Usage:

In [4]:
##### Request a GPU #####
## This function locates an available gpu for usage. In addition, this function reserves a specificed
## memory space exclusively for your account. The memory reservation prevents the decrement in computational
## speed when other users try to allocate memory on the same gpu in the shared systems, i.e., CADE machines. 
## Note: If you use your own system which has a GPU with less than 4GB of memory, remember to change the 
## specified mimimum memory.
def define_gpu_to_use(minimum_memory_mb = 3500):    
    thres_memory = 600 #
    gpu_to_use = None
    try: 
        os.environ['CUDA_VISIBLE_DEVICES']
        print('GPU already assigned before: ' + str(os.environ['CUDA_VISIBLE_DEVICES']))
        return
    except:
        pass
    
    torch.cuda.empty_cache()
    for i in range(16):
        free_memory = !nvidia-smi --query-gpu=memory.free -i $i --format=csv,nounits,noheader        
        if free_memory[0] == 'No devices were found':
            break
        free_memory = int(free_memory[0])
        
        if free_memory>minimum_memory_mb-thres_memory:
            gpu_to_use = i
            break
            
    if gpu_to_use is None:
        print('Could not find any GPU available with the required free memory of ' + str(minimum_memory_mb) \
              + 'MB. Please use a different system for this assignment.')
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_to_use)
        print('Chosen GPU: ' + str(gpu_to_use))
        x = torch.rand((256,1024,minimum_memory_mb-thres_memory)).cuda()
        x = torch.rand((1,1)).cuda()        
        del x
        
## Request a gpu and reserve the memory space
define_gpu_to_use()

Chosen GPU: 0


## + Define Utility Functions:

In [5]:
##### Preprocess Image #####
## This function is used to crop the largest 1:1 aspect ratio region of a given image.
## This is useful, especially for medical datasets, since many datasets have images
## with different aspect ratios and this is one way to standardize inputs' size.
class CropBiggestCenteredInscribedSquare(object):
    def __init__(self):
        pass

    def __call__(self, tensor):
        longer_side = min(tensor.size)
        horizontal_padding = (longer_side - tensor.size[0]) / 2
        vertical_padding = (longer_side - tensor.size[1]) / 2
        return tensor.crop(
            (
                -horizontal_padding,
                -vertical_padding,
                tensor.size[0] + horizontal_padding,
                tensor.size[1] + vertical_padding
            )
        )

    def __repr__(self):
        return self.__class__.__name__ + '()'

In [6]:
##### Split a dataset for training, validatation, and testing #####
## This function splits a given dataset into 3 subsets of 60%-20%-20% for train-val-test, respectively.
## This function is used internally in the dataset classes below.
def get_split(array_to_split, split):
    np.random.seed(0)
    np.random.shuffle(array_to_split)
    np.random.seed()
    if split == 'train':
        array_to_split = array_to_split[:int(len(array_to_split)*0.6)]
    elif split == 'val':
        array_to_split = array_to_split[int(len(array_to_split)*0.6):int(len(array_to_split)*0.8)]
    elif split == 'test':
        array_to_split = array_to_split[int(len(array_to_split)*0.8):]
    return array_to_split


In [7]:
##### Compute the number parameters (weights) #####
## This function computes the number of learnable parameters in a Pytorch model
def count_number_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Exercise 1 - MNIST Dataset and CNNs (Total of 32 points)
We begin this assignment by revisiting one of the problems from the previous assignment. Previously, we built a simple two-layer neural network to classify images containing hand-written digits. In this exercise, we are going to replace that two-layer neural network with a convolutional neural network. First, we load the images that we are going to work with into Pytorch Dataloader, and then define an evaluation metric for it.

In [8]:
##### Load MNIST dataset to Pytorch Dataloader #####
## Dowloand MNIST dataset train set from Pytorch (60,000 images total)
mnist_training_data = torchvision.datasets.MNIST(mnist_dataset_folder, train = True, \
                                        transform=transforms.ToTensor(), \
                                        target_transform=None, \
                                        download= True)
print('A summary of MNIST dataset:\n')
print(mnist_training_data,'\n')
## Dowloand MNIST dataset test set from Pytorch (10,000 images total)
mnist_test_data = torchvision.datasets.MNIST(mnist_dataset_folder, train = False, \
                                        transform=transforms.ToTensor(), \
                                        target_transform=None, \
                                        download= True)
print(mnist_test_data)
## Randomly split training images into 80%/20% for training/validation process
train_data_ex1, val_data_ex1 = torch.utils.data.random_split(mnist_training_data, \
                                    [int(0.8*len(mnist_training_data)),\
                                     len(mnist_training_data)-int(0.8*len(mnist_training_data))], \
                                     generator=torch.Generator().manual_seed(1))

assert(len(mnist_test_data) == 10000)
assert(len(train_data_ex1)+len(val_data_ex1) == 60000)

## Load files to Pytorch dataloader for training, validation, and testing
train_loader_ex1 = torch.utils.data.DataLoader(train_data_ex1, batch_size=16, shuffle=True, num_workers=8)
val_loader_ex1 = torch.utils.data.DataLoader(val_data_ex1, batch_size=128, shuffle=True, num_workers=8)
test_loader_ex1 = torch.utils.data.DataLoader(mnist_test_data, batch_size=128, shuffle=False, num_workers=8)

A summary of MNIST dataset:

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./deep_learning_datasets_ECE_6960_013/mnist
    Split: Train
    StandardTransform
Transform: ToTensor() 

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./deep_learning_datasets_ECE_6960_013/mnist
    Split: Test
    StandardTransform
Transform: ToTensor()


In [9]:
##### Compute accuracy for MNIST dataset #####
def get_accuracy_mnist(model, data_loader):
    ## Toggle model to eval mode
    model.eval()
    
    ## Iterate through the dataset and perform inference for each sample.
    ## Store inference results and target labels for AUC computation 
    with torch.no_grad():
        #run through several batches, does inference for each and store inference results
        # and store both target labels and inferenced scores
        acc = 0.0
        for image, target in data_loader:
            image = image.cuda(); target = target.cuda()
            probs = model(image)
            preds = torch.argmax(probs, 1)
            acc += torch.count_nonzero(preds == target)
                
        return acc/len(data_loader.dataset)
        

Now, your task is to implement a convolutional neural network, named ``model_ex1``, with the following order and set-up:
- Zero padding the image boundaries with 2 additional pixels
- Convolution Layer 1 - kernel size = 5; output features = 6; padding = 0; stride = 1; containing bias
- Relu Activation Function
- Max Pooling Layer 1 - kernel size = 2; stride = 2
- Convolution Layer 2 - kernel size = 5; output features = 16; padding = 0; stride = 1; containing bias
- Relu Activation Function
- Max Pooling Layer 2 - kernel size = 2; stride = 2
- Convolution Layer 3 - kernel size = 5; output features = 120; padding = 0; stride = 1; containing bias
- Relu Activation Function
- Fully-connected Layer 1 - input features = 120; output features = 84; containing bias
- Drop-out Layer - with dropout probability of 0.5
- Relu Activation Function
- Fully-connected Layer 2 - input features = 84; output features = 10; containing bias

In [10]:
##### Implement a CNN, named model_ex1, with the architecture specified above #####
### Your code starts here ###
import torch.nn as nn

class Flatten(nn.Module):
	def forward(self, x):
		return x.view(x.size(0), -1)

class model_ex1(nn.Module):
	def __init__(self):
		super(model_ex1, self).__init__()
  
		self.layers = nn.Sequential(
            nn.ZeroPad2d(2),
            
			nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=0, stride=1, bias=True),
			nn.ReLU(),
			nn.MaxPool2d(kernel_size=2, stride=2),
   
			nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, padding=0, stride=1, bias=True),
			nn.ReLU(),
			nn.MaxPool2d(kernel_size=2, stride=2),
   
			nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, padding=0, stride=1, bias=True),
			nn.ReLU(),
   
			Flatten(),
   
			nn.Linear(120, 84),
			nn.Dropout(0.5),
			nn.ReLU(),
   
			nn.Linear(84, 10)
			
		)
		
	def forward(self, x):
		return self.layers(x)

## Some Test Code
# m = model_ex1()
# input_ = torch.zeros(1,1,28,28)
# output_ = m(input_)

### Your code ends here ###

Next, you will implement the training process for ``model_ex1`` using stochastic gradient descent algorithm to optimize the softmax cross-entropy loss function. Your best trained model must meet the following requirements:
- Your best model should be named ``best_model_ex1``.
- Obtaining at least 98.5\% accuracy on the validation set.

In [11]:
from collections import defaultdict
def weight_init(module, initf):
	def foo(m):
		classname = m.__class__.__name__.lower()
		if isinstance(m, module):
			initf(m.weight)
	return foo

In [12]:
##### Training Process #####
### Your code starts here ###

DEVICE = "cuda:0"
model_ex1_1 = model_ex1()
model_ex1_1.to(DEVICE)
model_ex1_1.apply(weight_init(module=nn.Conv2d, initf=nn.init.kaiming_uniform_))
model_ex1_1.apply(weight_init(module=nn.Linear, initf=nn.init.kaiming_uniform_))
num_epochs = 100

optimizer = torch.optim.SGD(model_ex1_1.parameters(), lr=1e-2)
optimizer.zero_grad()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.99)
criterion_ce = nn.CrossEntropyLoss()
prev_acc = 0

save_dir = os.path.join(pre_folder, "save_model_ex1_1")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for epoch in range(num_epochs):
    torch.cuda.empty_cache()
    model_ex1_1.train()
    loss_dict = defaultdict(list)
    for image, labels in train_loader_ex1:
        optimizer.zero_grad()
        image = image.to(DEVICE)
        labels = labels.to(DEVICE)
        output = torch.softmax(model_ex1_1(image), dim=1)
        final_loss = criterion_ce(output, labels)
        final_loss.backward()
        optimizer.step()
        loss_dict["ce"].append(final_loss.detach().cpu())
    
    print(f'Train Loss after {epoch} epochs = {np.mean(np.array(loss_dict["ce"]))}')
    train_acc = get_accuracy_mnist(model_ex1_1, train_loader_ex1)
    val_acc = get_accuracy_mnist(model_ex1_1, val_loader_ex1)
    print(f'Train accuracy={train_acc}, Val accuracy={val_acc}')
    
    if val_acc > prev_acc:
        torch.save(model_ex1_1.state_dict(), os.path.join(save_dir, "model_best.torch"))
        prev_acc = val_acc
    
    if val_acc > 0.985:
        print("Accuracy of 98.5% reached - stopping model")
        best_model_ex1 = copy.deepcopy(model_ex1_1)
        break
    
    if epoch != 0 and epoch % 10 == 0:
        scheduler.step()
### Your code ends here ###

Train Loss after 0 epochs = 1.8888065814971924
Train accuracy=0.8492082953453064, Val accuracy=0.8505833148956299
Train Loss after 1 epochs = 1.6367112398147583
Train accuracy=0.8671249747276306, Val accuracy=0.8700833320617676
Train Loss after 2 epochs = 1.6131715774536133
Train accuracy=0.8733958005905151, Val accuracy=0.8733333349227905
Train Loss after 3 epochs = 1.583055019378662
Train accuracy=0.9586458206176758, Val accuracy=0.9589999914169312
Train Loss after 4 epochs = 1.522063970565796
Train accuracy=0.9644374847412109, Val accuracy=0.9667499661445618
Train Loss after 5 epochs = 1.5125211477279663
Train accuracy=0.9706458449363708, Val accuracy=0.9710833430290222
Train Loss after 6 epochs = 1.5068386793136597
Train accuracy=0.9728749990463257, Val accuracy=0.9716666340827942
Train Loss after 7 epochs = 1.5024651288986206
Train accuracy=0.9742082953453064, Val accuracy=0.9739999771118164
Train Loss after 8 epochs = 1.499698281288147
Train accuracy=0.9762916564941406, Val accur

Lastly, we are going to evaluate your best trained model on the test set.

In [13]:
##### Inference stage for MNIST dataset #####
test_acc = get_accuracy_mnist(best_model_ex1, test_loader_ex1)
print('MNIST Test Accuracy: %.3f%%' %(test_acc*100))

MNIST Test Accuracy: 98.640%


# Exercise 2 - ChestXray14 Dataset (Total of 68 points)
>### Brief explanation of the dataset
> The ChestXray14 dataset contains more than 100,000 frontal chest x-rays and labels for 14 different conditions. These labels were extracted from radiologists' reports associated with each image using natural language processing techniques. It was released at the end of 2017 and was the largest publicly available x-rays dataset for developing deep learning models at the time. More information about the ChestXray14 dataset can be found here: https://stanfordmlgroup.github.io/projects/chexnet/. 

In this exercise, we will be using only a subset of this dataset, 14,999 images in total. The labels of this dataset follow a multi-label structure, which means that each image can have more than one label. First, we need to obtain the dataset. Please download the file **Data_Entry_2017_v2020.csv** from https://nihcc.app.box.com/v/ChestXray-NIHCC and **image_names_chestxray14.csv** from Canvas, and put them in the same directory as this notebook.

In [ ]:
## Download part of the dataset if it was not downloaded yet and 
## put it to the xray14_dataset_folder folder

##### Report download status #####
def report_hook(count_so_far, block_size, total_size):
    current_percentage = (count_so_far * block_size * 100 // total_size)
    previous_percentage = ((count_so_far - 1) * block_size * 100 // total_size)
    if current_percentage != previous_percentage:
        sys.stdout.write('\r' + str((count_so_far * block_size * 100 // total_size)) \
                         + '% of download completed')
        sys.stdout.flush()

##### Download a subset of ChestXray14 dataset #####        
if xray14_dataset_folder != '/scratch/tmp/deep_learning_datasets_ECE_6960_013/chestxray14':
    os.makedirs(xray14_dataset_folder, exist_ok=True)
    from urllib.request import urlretrieve
    destination_file = xray14_dataset_folder + '/images_4.tar.gz'
    link = 'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz'
    if not os.path.isfile(destination_file):
        urlretrieve(link, destination_file, reporthook = report_hook)

    destination_file = xray14_dataset_folder + '/images_1.tar.gz'
    link = 'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz'
    if not os.path.isfile(destination_file):
        urlretrieve(link, destination_file, reporthook = report_hook)        
    
##### Extract the downloaded file #####
if xray14_dataset_folder != '/scratch/tmp/deep_learning_datasets_ECE_6960_013/chestxray14':
    destination_file = xray14_dataset_folder + '/images_4.tar.gz'
    tar = tarfile.open(destination_file, "r:gz")
    tar.extractall(path = xray14_dataset_folder)
    tar.close()
    destination_file = xray14_dataset_folder + '/images_1.tar.gz'
    tar = tarfile.open(destination_file, "r:gz")
    tar.extractall(path = xray14_dataset_folder)
    tar.close()

Next, we create a Pytorch class structure for this dataset. The dataset class is used to load, index, and preprocess samples for training, validation, and testing process.

In [ ]:
class Chestxray14Dataset(Dataset):
    ##### Initialize the class #####
    def __init__(self, path_dataset_folder, split = 'train'):
        ## Split parameter is used to specify which process the data is used for,
        ## and it can be 'train', 'val', and 'test'
        
        self.path_image_folder = path_dataset_folder + '/images'
        
        ## Get the filenames of all images in the dataset
        all_images_list = pd.read_csv('image_names_chestxray14.csv')

        ## Read the labels file which needs to be placed in the same folder as this notebook
        label_file = pd.read_csv('./Data_Entry_2017_v2020.csv')
        
        ## Merge labels and images information
        examples_to_use = pd.merge(all_images_list, label_file)
        
        ## This is the name of all possible labels in this dataset.
        ## The corresponding label of each sample is an array of 14 elements in which the elements are ordered
        ## in the same way as "self.set_of_finding_labels" and the value of each element represents the 
        ## presence of that condition in the sample. For example, if "cardiomegaly" and "pneumonia" are the two 
        ## conditions presence a given sample, then the corresponding label of that sample is represented 
        ## by an array  - [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
        self.set_of_finding_labels = ['Atelectasis', 'Cardiomegaly','Effusion',  'Infiltration', 'Mass',\
                                      'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation', 'Edema', \
                                      'Emphysema', 'Fibrosis','Pleural_Thickening', 'Hernia' ]
        
        ## Read labels from the label file
        examples_to_use['Finding Labels'] = examples_to_use['Finding Labels'].str.split(pat = '|')
        examples_to_use['Finding Labels'] = examples_to_use['Finding Labels'].apply(list).\
                                            to_frame(name='Finding Labels')
        for finding_label in self.set_of_finding_labels:
            examples_to_use[finding_label] = examples_to_use.apply(\
                                            lambda x: int(finding_label in x['Finding Labels']), axis=1)
        
        ## Get the list of all patient ids present in the dataset and split into
        ## training, validation and testing by patient id, but not by list of examples
        patient_ids = pd.unique(examples_to_use['Patient ID'])
        patient_ids = pd.DataFrame(get_split(patient_ids, split), columns = ['Patient ID'])
        
        ## Filter the examples to only use the ones that have the chosen patient ids
        examples_to_use = pd.merge(patient_ids,examples_to_use)        
        
        examples_to_use = examples_to_use[['Image Index'] + self.set_of_finding_labels]
        self.image_list = examples_to_use['Image Index'].values
        self.targets = examples_to_use[self.set_of_finding_labels].values
        
        ## Define data augmentation transformations for the input images. In this exercise, we use the following
        ## transformations: square center cropping, resizing to 224x224 (to be similar as ImageNet dataset), 
        ## converting to tensor, normalizing per channel (i.e., R, G, and B) 
        ## with the average and standard deviation of images in the ImageNet dataset        
        self.set_of_transforms = transforms.Compose(
        [CropBiggestCenteredInscribedSquare(),
         transforms.Resize(224),
         transforms.ToTensor(), 
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
        
    ##### Retrieve a sample with the corresponding index #####
    ## This function retrieve a sample from the dataset at the specified index 
    ## and returns an image and the corresponding label stored in Pytorch tensors     
    def __getitem__(self, index):
        curr_pil_image = Image.open(self.path_image_folder + '/' + self.image_list[index]).convert('RGB')
        image_to_return = self.set_of_transforms(curr_pil_image)
                
        return image_to_return, torch.FloatTensor(self.targets[index])
    
    ##### Get the length of the dataset #####
    def __len__(self):
        return len(self.image_list)
    
    ##### Access the name of conditions in the labels #####
    def get_labels_name(self):
        return self.set_of_finding_labels

We will then use the class function above to create structures for each training, validation, and test set.

In [ ]:
## Create datasets for this exercise
train_dataset_ex2 = Chestxray14Dataset(xray14_dataset_folder)
val_dataset_ex2 = Chestxray14Dataset(xray14_dataset_folder, split = 'val')
test_dataset_ex2 = Chestxray14Dataset(xray14_dataset_folder, split = 'test')


## Please contact the TA if any of the assertions fails
assert(len(train_dataset_ex2) == 8837)
assert(len(val_dataset_ex2) == 2924)
assert(len(test_dataset_ex2) == 3238)
assert(np.sum(train_dataset_ex2.targets)==5893)
assert(np.sum(train_dataset_ex2.targets[:,7])==404)
assert(np.sum(val_dataset_ex2.targets)==1810)

Now, we report a statistical summary of the training set to get an idea of what this dataset looks like. We also show a sample of this dataset to get ourselves familiar with the type of images we are working with in this exercise. 

In [ ]:
##### Helper function for display text below #####
def join_str_array_to_labels(str_array,labels):
    return ','.join(['\n{}: {}'.format(labels[index_element], str_array_element) 
                for index_element, str_array_element in enumerate(str_array)])

## Show the statistics of training set
frequencies = np.sum(train_dataset_ex2.targets, axis = 0)/len(train_dataset_ex2)
text_frequencies = ['{:.2f}%'.format(frequency*100) for frequency in frequencies]                    
print('Percentage of positive examples in each class in the training set: ')
print(join_str_array_to_labels(text_frequencies, train_dataset_ex2.get_labels_name()))

## Plot a sample from the training set
print('\n\nShowing one example from the dataset:')
plt.imshow(train_dataset_ex2[1][0].cpu().numpy()[0,:,:], cmap = 'gray')
print(join_str_array_to_labels(train_dataset_ex2[1][1],train_dataset_ex2.get_labels_name()))


For this exercise, we will use AUC (also called AUROC) metric to evaluate the performance of a model. The AUC metric is defined as the area under the Receiver Operating Characteristic (ROC) curve, and has a value between 0 and 1. An AUC of 0.5 is what a model producing random outputs can maximally achieve. The higher the AUC is the better the model. In medical related applications, we want to avoid false negatives or false positives outcomes. Thus, the ROC curve is frequently used because it measures the trade-off between false positives and false negatives (sensitivity and specificity to be precise) of a model. Besides, ROC is insensitive to imbalanced datasets. We will define this metric in the cell below.

In [ ]:
##### Calculate AUC metric #####
## This function compute AUC from the given input arrays i.e., predicted value and ground truth arrays
def auroc(logits_predicted, target):
    fpr, tpr, _ = roc_curve(target, logits_predicted)
    return auc(fpr, tpr)

##### Compute AUC of a given dataset #####
## This function takes a model and Pytorch data loader as input. 
## The given model is used to predict the expected label for each sample in the Pytorch data loader. The 
## model output for each sample is an array with 14 elements corresponding with 14 conditions in the 
## ChestXray14 dataset. Then, the AUC is computed for each condition.
def get_score_model_chestxray_binary_model(model, data_loader):
    ## Toggle model to eval mode
    model.eval()
    
    ## Iterate through the dataset and perform inference for each sample.
    ## Store inference results and target labels for AUC computation 
    with torch.no_grad():
        
        logits_predicted = np.zeros([0, 14])
        targets = np.zeros([0, 14])
        ## Iterate through the dataset and perform inference for each sample.
        ## Store inference results and target labels for AUC computation  
        for image, target in data_loader:
            image = image.cuda()
            logit_predicted = model(image)
            logits_predicted = np.concatenate((logits_predicted, logit_predicted.cpu().detach().numpy())\
                                              , axis = 0)
            targets = np.concatenate((targets, target.cpu().detach().numpy()), axis = 0)
            
    ## Return a list of auc values in which each value corresponds to one of the 14 labels
    return [auroc(logits_predicted[:,i], targets[:,i]) for i in range(14)]

## Exercise 2.1 - Adopt an ImageNet Pretrained Model (34 points)
In this section, you will adopt ResNet-18 pretrained on Imagenet dataset provided in the torchvision package from Pytorch library (https://pytorch.org/docs/stable/torchvision/models.html#id3) for ChestXray14 dataset. To obtain that goal, our tasks are as follows:
- Setting up Pytorch dataloader for training, validation, and test sets
- Modifying the ResNet-18 model to have 14 neurons, which corresponds to 14 labels, in the output layer
- Selecting a loss function and justify your choice

In [ ]:
##### Set-up Pytorch dataloader #####
### Your code starts here ###


### Your code ends here ###

In [ ]:
##### Modify ResNet-18 #####
### Your code starts here ###


### Your code ends here ###

**Write a short reasoning for your loss function selection:**

In [ ]:
##### Define loss function #####
### Your code starts here ###


### Your code ends here ###

Now, you will implement the training process for the modified ResNet-18 model. Your best trained model must achieve the mean AUC of 14 classes of at least 0.725 on the validation set.

In [ ]:
##### Training Process #####
### Your code starts here ###

    
### Your code ends here ###

After obtaining the desired accuracy on the validation set, test your best model on the test set, and specify the anomalies/labels of which your model achieves best and worst AUC score.

In [ ]:
##### Inference stage for ChestXray14 dataset #####
### Your code starts here ###


### Your code ends here ###

## Exercise 2.2 - Implement Your Own Model (34 points) 
For this exercise, you are going to build your own model for ChestXray14 dataset. Your model has to satisfy the following conditions:
- Containing at most 500,000 learnable parameters.
- Training your model from scratch, i.e., none of the learnable parameters in your model is extracted from another pretrained model.
- Obtaining at least 0.67 for the mean AUC of 14 classes on the validation set.

In addition to having at least one model meets the requirements above, you need to show that you have tried:
- at least 2 other architectures, i.e., with different number of layers, different number of feature maps at each layers, different combinations of layers, etc.
- at least 2 hyperpameters with a set of values for each hyperparameter.

Describe/Analyze the experiments you perform with plots, tables, and text on what works well and what does not. Please also include the code associated with these experiments.

In [ ]:
##### Implement Your Own Model #####
### Your code starts here ###

    
### Your code ends here ###

Now, we will verify if your best model meets the number of learnable parameters requirement.

In [ ]:
##### Verify the number of learnable parameters requirement #####
## ***** Please change the parameter inside the "count_number_parameters" 
##       to the name of the model you want to test *****
if count_number_parameters(best_model_ex22) > 500000:
    print('Warning! Your model exceeds the learnable parameters requirement!')

Let's test your best model on the test set!

In [ ]:
##### Perform inference with your best model #####
### Your code starts here 


### Your code ends here